<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/MobileNet_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import absolute_import, division
try: 
    import tensorflow as tf
    print(f'tensorflow: {tf.__version__}')
except ModuleNotFoundError as err:
    %pip install tensorflow
    try: 
        import tensorflow as tf
        print(f'\ntensorflow: {tf.__version__}')
    except Exception as err:
        print(err)
        
import glob, os, shutil
from os.path import *
import numpy as np
from google.colab import drive, files

contentPth = os.getcwd()
myDrivePth = '/content/drive/My Drive'
deletePth = '/content/sample_data'
if exists(deletePth):
    shutil.rmtree(deletePth)
    deletePth=None

drive.mount('/content/drive', force_remount=True)

os.chdir(myDrivePth)
import ImportDriveFiles
from BashColors import C


mobilenet_v2=None

In [ ]:
imgPth = join(myDrivePth, 'Tensorflow Images')
if exists(imgPth):
    print(imgPth)
    fileList = glob.glob('**', recursive=True)
for fil in sorted(fileList):
    print(fil)
len(fileList)

In [ ]:
physicalDevices = tf.config.experimental.list_physical_devices('GPU')
print(f'{C.Blue}{len(physicalDevices)} gpu available')
visableDevices=tf.config.get_visible_devices()
for device in visableDevices:
    print(f'{device.device_type}: {device}')
tf.config.experimental.set_memory_growth(physicalDevices[0], True)

In [ ]:
mobilenet_v2 = tf.keras.applications.MobileNetV2(input_shape=None,
                                                 alpha=1.0,
                                                 include_top=True,
                                                 weights=None,
                                                 input_tensor=None, 
                                                 pooling=None,
                                                 classes=1000,
                                                 classifier_activation='softmax')

mobilenet_v2.compile()

modelName = mobilenet_v2.name + '_no_weights.h5'

print(f'saved: {C.IBlue}{modelName}')
mobilenet_v2.save(modelName)

In [ ]:
tfModelsPth = '/content/drive/My Drive/Tensorflow Models'
sourcePth = join(contentPth, modelName)
destinationPth = join(tfModelsPth, modelName)
if exists(sourcePth) and exists(tfModelsPth):
    print(sourcePth)
    print(destinationPth)
    shutil.copy2(sourcePth, destinationPth)

files.view(tfModelsPth)

In [ ]:
from tensorflow import keras
re_model=keras.models.load_model(destinationPth)
re_model.compile()


re_model

In [ ]:
from tensorflow.keras.preprocessing import image

In [ ]:
from IPython.display import Image, display
from tensorflow.keras.applications import imagenet_utils
import numpy as np

imageList=None
imgPth = '/content/drive/My Drive/Test Images'
if exists(imgPth):
    print(imgPth)
    os.chdir(imgPth)
    imageList=glob.glob('**')
    os.chdir(contentPth)

counter=0
for thisImage in sorted(imageList):
    counter+=1
    thisImage=join(imgPth, thisImage)
    img = image.load_img(thisImage, target_size=(224,224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    result=tf.keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)
    print(f'{counter}. {C.IBlue}{basename(thisImage)}')
    show=Image(filename=thisImage, width=224, height=224)
    display(show)

    thisPrediction = re_model.predict(result)
    thisResult = imagenet_utils.decode_predictions(thisPrediction)
    printResult(inputResult=thisResult)
    _ = thisPrediction.max()
    percent=thisPrediction.max()
    print(f'{C.IBlue}{percent * 100} percent')
    print(f'{C.IPurple}{thisResult[0]}{C.ColorOff}\n')


In [ ]:

def printResult(inputResult):
    for item in result[0]:
        percent=item[2] * 100
        print(f'probability: {percent}%\t {item[1]}')

In [ ]:
import tensorflow_datasets as tfds
tfds.list_builders()

In [ ]:
import tensorflow_datasets as tfds
builder = tfds.builder('fashion_mnist')
info = builder.info
print(info.name)
print(info.full_name)

In [ ]:
data,info = tfds.load('fashion_mnist', split='train', with_info=True)
# tfds.visualization.show_examples(ds=data ,ds_info=info )
print(f'data: {len(data)}')
print(f'info: {info}')
tfds.as_dataframe(data.take(9), info)

In [ ]:
sourcePth = join(info.data_dir)
destinationPth = join(contentPth, info.full_name)
if not exists(destinationPth):
    shutil.copytree(sourcePth, destinationPth)

os.chdir(destinationPth)
higgsStuff = glob.glob('**', recursive=True)
for item in sorted(higgsStuff):
    print(f'{C.IWhite}{item}')


In [ ]:

sourcePth = destinationPth
print(f'sourcePth: {sourcePth}')
datasetsPth = '/content/drive/My Drive/Tensorflow Datasets/'
datasetsDirectoryPth = join(datasetsPth, info.full_name)

if not exists(datasetsDirectoryPth):
    print(f'copy to {datasetsDirectoryPth}')
    shutil.copytree(sourcePth, datasetsDirectoryPth)
    files.view(datasetsDirectoryPth)
else: print(f'{datasetsDirectoryPth} exists')

In [ ]:
files.view(datasetsDirectoryPth)

In [ ]:
fig = tfds.show_examples(data, info)
fig